In [1]:
# Importing Libraries

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [35]:
labels = labels=['WALKING','WALKING_UPSTAIRS','WALKING_DOWNSTAIRS','SITTING','STANDING','LAYING']

### Data

In [5]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [6]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [19]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).values
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [20]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).values

In [21]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [22]:
# Importing tensorflow
np.random.seed(42)
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.set_random_seed(42)

In [23]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [24]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [25]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [26]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [27]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [29]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [30]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [31]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                5376      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 6)                 198       
                                                                 
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [33]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples


2023-01-21 17:16:34.563025: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


Epoch 1/30
7344/7352 [============================>.] - ETA: 0s - loss: 1.2153 - acc: 0.5018

/home/ankit/.local/lib/python3.8/site-packages/keras/engine/training_v1.py:2332: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


7352/7352 [==============================] - 14s 2ms/sample - loss: 1.2150 - acc: 0.5019 - val_loss: 0.9765 - val_acc: 0.5636
Epoch 2/30
7352/7352 [==============================] - 14s 2ms/sample - loss: 0.7382 - acc: 0.6982 - val_loss: 0.7342 - val_acc: 0.6790
Epoch 3/30
7352/7352 [==============================] - 14s 2ms/sample - loss: 0.7690 - acc: 0.6915 - val_loss: 0.6847 - val_acc: 0.7136
Epoch 4/30
7352/7352 [==============================] - 14s 2ms/sample - loss: 0.7299 - acc: 0.7282 - val_loss: 0.7027 - val_acc: 0.7306
Epoch 5/30
7352/7352 [==============================] - 14s 2ms/sample - loss: 0.5722 - acc: 0.7909 - val_loss: 0.5887 - val_acc: 0.7937
Epoch 6/30
7352/7352 [==============================] - 14s 2ms/sample - loss: 0.4494 - acc: 0.8449 - val_loss: 0.5355 - val_acc: 0.7900
Epoch 7/30
7352/7352 [==============================] - 15s 2ms/sample - loss: 0.3807 - acc: 0.8818 - val_loss: 0.4537 - val_acc: 0.8660
Epoch 8/30
7352/7352 [==============================

In [38]:
# Confusion Matrix
from mlxtend.plotting import plot_confusion_matrix
cm = confusion_matrix(Y_test, model.predict(X_test))
print(cm)

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        24        0                   0   
SITTING                  1      370       118        0                   0   
STANDING                 0       93       434        1                   0   
WALKING                  0        3         0      476                   9   
WALKING_DOWNSTAIRS       0        0         0       11                 408   
WALKING_UPSTAIRS         0        1         1        5                   1   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             3  
SITTING                            2  
STANDING                           4  
WALKING                            8  
WALKING_DOWNSTAIRS                 1  
WALKING_UPSTAIRS                 463  


In [39]:
score = model.evaluate(X_test, Y_test)

In [40]:
score

[0.30736647124741084, 0.90295213]

- With a simple 2 layer architecture we got 90.03% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning